Import statements

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
header_list = ['SpMax_L', 'J_Dz(e)', 'nHM', 'F01[N-N]', 'F04[C-N]','NssssC', 'nCb-', 'C%', 'nCp', 'nO', 'F03[C-N]',
               'SdssC', 'HyWi_B(m)', 'LOC', 'SM6_L', 'F03[C-O]', 'Me', 'Mi', 'nN-N', 'nArNO2', 'nCRX3', 'SpPosA_B(p)', 
              'nCIR', 'B01[C-Br]', 'B03[C-Cl]', 'N-073', 'SpMax_A', 'Psi_i_1d', 'B04[C-Br]', 'SdO' , 'TI2_L', 'nCrt',
               'C-026', 'F02[C-N]', 'nHDon', 'SpMax_B(m)', 'Psi_i_A', 'nN', 'SM6_B(m)', 'nArCOOR', 'nX', 'TARGET']
df = pd.read_csv("NewBioDeg.csv", names = header_list)
columns = df.columns
df.head()

FileNotFoundError: ignored

Change target class labels to '0' for 'NRB' and '1' for 'RB'. There are almost twice as many '0' than '1', so stratifying the data when doing the train-test split is necessary.

In [ ]:
df.replace('RB', 1, inplace=True)
df.replace('NRB', 0, inplace=True)
print(df['TARGET'].value_counts())

0    699
1    356
Name: TARGET, dtype: int64


Now let's look at the missing values in the data. If we drop columns with any missing data altogether, we only have 787 out of 1055 examples to work with. Instead, we decide to impute the data. Note that there are both categorical/binary and numerical features. We try to impute the categorical ones with (???) and numerical ones with the mean.

In [ ]:
for c in columns:
    print(c + ": " + str(len(df[c].unique())))
print("Original number of examples: " + str(len(df)))
print("After dropping NAN: " + str(len(df.dropna())))
df['SpMax_L'].fillna(df['SpMax_L'].mean(), inplace=True)

SpMax_L: 441
J_Dz(e): 1016
nHM: 12
F01[N-N]: 5
F04[C-N]: 16
NssssC: 14
nCb-: 16
C%: 189
nCp: 16
nO: 13
F03[C-N]: 22
SdssC: 383
HyWi_B(m): 752
LOC: 372
SM6_L: 510
F03[C-O]: 25
Me: 168
Mi: 126
nN-N: 4
nArNO2: 5
nCRX3: 5
SpPosA_B(p): 352
nCIR: 14
B01[C-Br]: 3
B03[C-Cl]: 3
N-073: 5
SpMax_A: 330
Psi_i_1d: 206
B04[C-Br]: 3
SdO: 470
TI2_L: 551
nCrt: 9
C-026: 12
F02[C-N]: 16
nHDon: 9
SpMax_B(m): 702
Psi_i_A: 623
nN: 9
SM6_B(m): 860
nArCOOR: 6
nX: 18
TARGET: 2
Original number of examples: 1055
After dropping NAN: 787


Checking again for null values.

In [ ]:
for c in columns:
    print(c, df[c].isna().sum())
df.dropna()

SpMax_L 0
J_Dz(e) 7
nHM 5
F01[N-N] 8
F04[C-N] 5
NssssC 9
nCb- 5
C% 5
nCp 10
nO 8
F03[C-N] 6
SdssC 3
HyWi_B(m) 7
LOC 10
SM6_L 2
F03[C-O] 7
Me 13
Mi 9
nN-N 12
nArNO2 10
nCRX3 4
SpPosA_B(p) 4
nCIR 4
B01[C-Br] 9
B03[C-Cl] 10
N-073 7
SpMax_A 2
Psi_i_1d 8
B04[C-Br] 5
SdO 7
TI2_L 8
nCrt 1
C-026 5
F02[C-N] 5
nHDon 7
SpMax_B(m) 8
Psi_i_A 4
nN 3
SM6_B(m) 4
nArCOOR 4
nX 9
TARGET 0


,SpMax_L,J_Dz(e),nHM,F01[N-N],F04[C-N],NssssC,nCb-,C%,nCp,nO,...,C-026,F02[C-N],nHDon,SpMax_B(m),Psi_i_A,nN,SM6_B(m),nArCOOR,nX,TARGET
0,3.919,2.6909,0.0,0.0,0.0,0.0,0.0,31.4,2.0,0.0,...,0.0,0.0,0.0,2.949,1.591,0.0,7.253,0.0,0.0,1
1,4.170,2.1144,0.0,0.0,0.0,0.0,0.0,30.8,1.0,1.0,...,0.0,0.0,0.0,3.315,1.967,0.0,7.257,0.0,0.0,1
2,3.932,3.2512,0.0,0.0,0.0,0.0,0.0,26.7,2.0,4.0,...,0.0,0.0,1.0,3.076,2.417,0.0,7.601,0.0,0.0,1
5,4.236,3.4286,0.0,0.0,0.0,0.0,0.0,28.6,2.0,4.0,...,0.0,0.0,0.0,3.351,2.556,0.0,7.904,0.0,0.0,1
8,4.596,3.0777,0.0,0.0,0.0,0.0,2.0,44.4,2.0,0.0,...,0.0,0.0,0.0,3.626,1.917,0.0,7.939,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,4.807,3.3179,1.0,0.0,0.0,0.0,3.0,43.8,1.0,1.0,...,2.0,0.0,1.0,3.926,2.568,0.0,8.562,0.0,1.0,0
1048,5.265,3.3444,2.0,0.0,6.0,0.0,1.0,35.3,2.0,3.0,...,0.0,3.0,0.0,4.698,2.463,2.0,9.666,0.0,0.0,0
1049,5.029,2.5966,0.0,0.0,0.0,0.0,4.0,46.7,0.0,4.0,...,0.0,0.0,0.0,3.919,2.611,0.0,8.958,2.0,0.0,0
1051,5.287,3.3732,0.0,0.0,9.0,0.0,0.0,35.3,0.0,9.0,...,0.0,3.0,0.0,3.787,3.083,3.0,9.278,0.0,0.0,0


In [ ]:
corr = df.corr()
sns.heatmap(corr, annot = True)
plt.show()